In [3]:
import pandas as pd
import numpy as np
import re, glob

# Tsv to data frame

In [4]:

filename = '/home/mhiraga/CorpusData_New/Corpus_Data/Annotated_ALL_Shina_Hira_2/S163220F.txt.tsv'
df = pd.read_csv('/home/mhiraga/CorpusData_New/Corpus_Data/Annotated_ALL_Shina_Hira_2/20216S04.txt.tsv', 
                     sep='\t', skiprows=8, comment='#',header=None,skip_blank_lines=True,encoding='utf-8') #index_col=0 # to set the index to column 0

In [6]:
#df[:100]


# Feature extraction

Extract features

Using informative linguistic features can improve a machine learning model. I investigated which features were most effective for the automatic annotation using the corpus data. I used various types of features, including POS tags and lexical items in a sliding window around the target particle, which have been used by previous English preposition \citep{tetreault2008ups} and Korean particle error detection systems \citep{,RossIsraelDiss}. I also included dependency relations used by Japanese particle error detection systems \citep{suzuki2006learning,oyama2016automatic}. 
I run non-exhaustive feature selection experiments on the development sets for two particle groups, {\it de} and {\it ni}. I chose these particle groups because the group {\it de} has similar class distributions with other particles, and the {\it ni} experiment contains relatively more distributed classes than other particles. Below is the list of features that are included in the feature selection experiments.

Target Particle
Word (window size -3 to +3)
POS   (window size -3 to +3)
Lemmas (window size -3 to +3)
Category of the nouns selected in \ref{featureword}
Dependency head of the particle
All the other particles in the same sentence
All the verbs and auxiliary verbs that appear after the target particle

The experiments were done manually by unselecting some of the features.


In [17]:
import pandas as pd
import numpy as np
import re, glob
from sklearn import preprocessing
import math

def getBunrui(word,POS):
    
    word_cat_N = set()
    
    if POS.startswith('名詞'):
        try:
            word_cat_N_list = bunrui[bunrui[12].str.match(r'^'+word+r'\b')][5].to_numpy() 
            if len(word_cat_N_list) != 0:
                word_cat_N_list[0] = word_cat_N_list[0].reset_index()
                return word_cat_N_list[0]
            else:
                
                return "0"
        except:
            return "None"

    
def labelReplace(wordlist): #replace words
    wordlist = list(filter(('*').__ne__, wordlist))
    wordlist = list(filter(('').__ne__, wordlist))
    wordlist = list(filter(('_').__ne__, wordlist))
    wordlist = list(filter(('-').__ne__, wordlist))
    wordlist = [i for i in wordlist if not i in ['nan', np.nan]]
    
    for i in range(len(wordlist)):
        #print(item)
        
        if str(wordlist[i]).startswith("Top"):

            wordlist[i] = 'Topic'

        elif str(wordlist[i]) in ['NewSub', 'SubUnknown']:
            wordlist[i] = 'Sub'
        elif str(wordlist[i]) == 'Unkown':
            wordlist[i] = 'Unknown'
        elif str(wordlist[i]) == 'V-Mod':
            wordlist[i] = 'VMod'
        elif str(wordlist[i]) == 'Comp':
            wordlist[i] = 'Quote/Comp'
        elif str(wordlist[i])== 'Quote':
            wordlist[i] = 'Quote/Comp'

        elif str(wordlist[i]) == 'Obj-Ga':
            wordlist[i] = 'Ga-Obj'
        elif str(wordlist[i]) == 'NiObj':
            wordlist[i] = 'NiID'
        elif str(wordlist[i]) == 'NiNaru':
            wordlist[i] = 'NiChange'
    return wordlist

def getNeighboringWords(filename, df, prt):  #get Neighboring words and their POS
    
    classlabel= []
    correctprt = []
    list_of_dict = []
    feature_dict={}
    target_word = {}
    ## for addition error, there's no particles marked "correct". 
    ## Therefore, it is classification of finding which particle should be "NULL" or not, and if "incorrect", what function.
    ## prt is a list of all particles here.
    for index, row in df.iterrows():
        feature_dict={}
        
        if (str(df.loc[index][5]) in [prt]) and str(df.loc[index][3]).split('\\')[0].replace('feature=','').startswith('助詞') and str(df.loc[index][6]) in ['Incorrect', 'Correct', 'Attn']:
            if df.loc[index][6] == 'Correct':
                
                function_label = list(df.loc[index][9:20])
                function_label = labelReplace(function_label)
                function_label.append('COR')
                
                #classlabel.append(str(function_label)) 
                classlabel.append(function_label)
            elif df.loc[index][6] in ['Incorrect','Attn']:
                
                function_label = list(df.loc[index][9:20])
                function_label = labelReplace(function_label)
                if len(function_label) > 0 and function_label[0] == "Unknown":
                    if str(df.loc[index][7]).startswith(r"[A-Za-z]") == "False" or df.loc[index][7] == "nan" or pd.isna(df.loc[index][7]): # or df.loc[index][7] == np.nan :
                        function_label.append("Other")
                    else:
                        function_label.append(df.loc[index][7])
                function_label.append('INC')
                #classlabel.append(str(function_label)) # for multi class 
                classlabel.append(function_label)
                
            if df.loc[index][20] == 'D':
                dephead_id = str(df.loc[index,22])
                feature_dict['dep_head'] = df.loc[df[0]==dephead_id,5].values[0]
                    
                
                    #print('22:',filename)
            elif df.loc[index][22] == 'D':
                dephead_id = str(df.loc[index,24])
                feature_dict['dep_head'] = df.loc[df[0]==dephead_id,5].values[0]
                    #print('24:',filename)
           
                
            target_word[filename.replace('~/Documents/','')+'_'+str(df.loc[index][0])]=feature_dict
            feature_dict['target']=str(df.loc[index][2])

            
            ## if target word is a particle, 
            ## get other particles in the sentence
            ## otherwise, 'NONE'
            target_word_id = str(df.loc[index][0])
            same_sent_id = target_word_id.split('-')[0]

            if (str(df.loc[index][5]) in prt) and str(df.loc[index][3]).split('\\')[0].replace('feature=','').startswith('助詞'):
                
                same_sentence = df.loc[df[0].str.contains('^'+same_sent_id+'-')==True]
                
                prt_list = []
                verb_list = []
                for i, row_i in same_sentence.iterrows():
                    if index != i: 
                       
                        if (str(df.loc[i][5]) in prt) and str(df.loc[i][3]).split('\\')[0].replace('feature=','').startswith('助詞'):
                            prt_list.append(same_sentence.loc[i][5])
                    #else:
                    #    print(index, i)

                    if str(same_sentence.loc[i][3]).split('\\')[0].replace('feature=','').startswith('動詞'):
                        verb_list.append(same_sentence.loc[i][5]) # verb lemma
                
                for i in range(len(prt_list)):
                    feature_dict['OtherPrt_'+str(i)] = prt_list[i]
                
                for j in range(len(verb_list)):
                    feature_dict['Verbs_'+str(j)] = verb_list[j]
                

                
            ##get dependency head
            
            if df.loc[index][20] == 'D':
                dephead_id = str(df.loc[index,22])
                feature_dict['dep_head'] = df.loc[df[0]==dephead_id,5].values[0]
                
            elif df.loc[index][22] == 'D':
                dephead_id = str(df.loc[index,24])
                feature_dict['dep_head'] = df.loc[df[0]==dephead_id,5].values[0]
            
            
           #get words and POS in the sliding window -3 to +3
            if index >= df.shape[0]-df.shape[0]+1:
                word = str(df.loc[index-1][2])
              
                feature_dict['word-1']=str(df.loc[index-1][2])
                feature_dict['lemma-1'] = str(df.loc[index-1][5])
                POS = str(df.loc[index-1][3]).split('\\')[0].replace('feature=','').split(',')[0]
                feature_dict['POS-1']=str(df.loc[index-1][3]).split('\\')[0].replace('feature=','').split(',')[0]
                
                #feature_dict['wordcat-1'] = getBunrui(word,POS) #this was not in the final version #this doesn't work
                #getBunrui method didn't work, so below is code to get "wordcat" feature.
                
                if POS.startswith('名詞'):
                    try:
                        word_cat_N_list = bunrui[bunrui[12].str.match(r'^'+word+r'\b')][5].to_numpy() #get_values() #r'^'+word+r'\b'
                        if len(word_cat_N_list) != 0:
                            feature_dict['wordcat-1'] = word_cat_N_list[0]
                    except:
                        pass
                
                
                    
                    
                        
                if index >= df.shape[0]-df.shape[0]+2:
                    
                    word = str(df.loc[index-2][2]) 
                    feature_dict['word-2']=word 
                    #feature_dict['lemma-2'] = str(df.loc[index-2][5]) # not in the final version
                    POS = str(df.loc[index-2][3]).split('\\')[0].replace('feature=','').split(',')[0]
                    feature_dict['POS-2']=POS
                    
                    #feature_dict['wordcat-2'] = getBunrui(word,POS) # not in the final version
                    
                    if index >= df.shape[0]-df.shape[0]+3:
                  
                        feature_dict['word-3']=str(df.loc[index-3][2])
                        #feature_dict['lemma-3'] = str(df.loc[index-3][5])
                        feature_dict['POS-3']=str(df.loc[index-3][3]).split('\\')[0].replace('feature=','').split(',')[0]
                   

            if index < df.shape[0]-1:
                word = str(df.loc[index+1][2])
                feature_dict['word+1']=word
                feature_dict['lemma+1'] = str(df.loc[index+1][5]) #not in the final
                POS = str(df.loc[index+1][3]).split('\\')[0].replace('feature=','').split(',')[0]
                feature_dict['POS+1']=POS
                #feature_dict['wordcat+1'] = getBunrui(word,POS) #not in the final
                
                if index < df.shape[0]-2:
                    pass
                    feature_dict['word+2']=str(df.loc[index+2][2])
                    #feature_dict['lemma+2'] = str(df.loc[index+2][5])
                    feature_dict['POS+2']=str(df.loc[index+2][3]).split('\\')[0].replace('feature=','').split(',')[0]

                    if index < df.shape[0]-3:
          
                        feature_dict['word+3']=str(df.loc[index+3][2])
                        #feature_dict['lemma+3'] = str(df.loc[index+3][5])
                        feature_dict['POS+3']=str(df.loc[index+3][3]).split('\\')[0].replace('feature=','').split(',')[0]
               
            list_of_dict.append(feature_dict)
    
  
    return list_of_dict, classlabel #for multi_class classification
    

def put_together(path,prt):
    everything = []
    all_labels = []
   
    
    files = [str(pp) for pp in glob.glob(path)]   
  
    for file in files:
        
        df = pd.read_csv(file,sep='\t', skiprows=8, comment='#',header=None,keep_default_na=False, skip_blank_lines=True,encoding='utf-8') #index_col=0 # to set the index to column 0
        
        list_of_dict, labels = getNeighboringWords(file,df,prt)  
        
        everything += list_of_dict
        all_labels += labels
    
    return everything, all_labels
        
def put_together_1(path,prt_list): #prt_list instead of prt
    everything = []
    all_labels = []
  
    
    files = glob.glob(path)   
    
    for file in files:
        df = pd.read_csv(file,sep='\t',skiprows=8, comment='#',header=None,keep_default_na=False,skip_blank_lines=True,encoding='utf-8') #index_col=0 # to set the index to column 0
        list_of_dict, labels = getNeighboringWords(file,df,prt_list)  
        everything += list_of_dict
        all_labels += labels
    
    return everything, all_labels        





In [18]:
##train
all_prt = ['に','で','を','は','が','と','も','の','NULL','Other']

## running this will take a while
path = '/home/mhiraga/CorpusData_New/Corpus_Data/experiment/train/*'

everything_ni, all_labels_ni = put_together(path,'に')    



In [19]:
everything_de, all_labels_de = put_together(path,'で')
everything_o, all_labels_o = put_together(path,'を')
everything_wa, all_labels_wa = put_together(path,'は')
everything_ga, all_labels_ga = put_together(path,'が')
everything_to, all_labels_to = put_together(path,'と')
everything_mo, all_labels_mo = put_together(path,'も')
everything_no, all_labels_no = put_together(path,'の')
everything_other, all_labels_other = put_together(path,'Other')





In [20]:
everything_null, all_labels_null = put_together(path,str('NULL'))
print(len(everything_null))



0


In [21]:
def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements '''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

print(len(everything_ni),len(all_labels_ni))




2178 2178


# Create Synthetic Data from "Correct" samples

In [22]:
import copy
def copied_together(everything,all_labels,all_prt, prt): #everything = original list of dict with labels, all_labels=class labels
    
    copied = {}
    copied_labels = copy.deepcopy(all_labels)
    
    copied_labels_frame = pd.DataFrame({"class": copied_labels})
    copied_all_labels = {}
    inc_prtlist= [i for i in all_prt if i not in [prt]]

    for i in inc_prtlist:
        #print(i)
        copied_df = pd.DataFrame(everything) # for each particle, create data frame from dict "everything"
        copied_df = copied_df.join(copied_labels_frame)
        
       
        
        copied_df.loc[(copied_df["target"] == prt),"target"] = i
        
        
        for item in copied_df["class"]:
        
            if len(item) > 1:
                if item[1] == 'INC':
                    copied_df.drop(copied_df[copied_df["class"].isin([item])].index, inplace=True)
                             
                elif len(item) == 3:
                    item[2] = 'INC'
                    
                else:
                    item[1] = 'INC' 
                #print(item)
        
        copied_list = copied_df.fillna('none').to_dict('records')
    

        copied_labels = copied_df["class"]
        copied_labels1 = copied_labels.to_list()
        copied_all_labels[i] = copied_labels1
        copied_list = copied_df.drop("class",axis=1)
        copied_list1 = copied_list.fillna('none').to_dict('records')
        
        copied[i] = copied_list1 # put df for each incorrect particle in a dict
        
    return copied, copied_all_labels
        

    


In [23]:

all_prt = ['に','で', 'を','は','が','と','も','の','NULL','Other']


prt = 'に'
final_ni, final_labels_ni = copied_together(everything_ni,all_labels_ni,all_prt,prt)

prt = 'で'
final_de, final_labels_de = copied_together(everything_de,all_labels_de,all_prt,prt)

prt = 'を'
final_o, final_labels_o = copied_together(everything_o,all_labels_o,all_prt,prt)

prt = 'は'
final_wa, final_labels_wa = copied_together(everything_wa,all_labels_wa,all_prt,prt)

prt = 'が'
final_ga, final_labels_ga = copied_together(everything_ga,all_labels_ga,all_prt,prt)

prt = 'と'
final_to, final_labels_to = copied_together(everything_to,all_labels_to,all_prt,prt)

prt = 'も'
final_mo, final_labels_mo = copied_together(everything_mo,all_labels_mo,all_prt,prt)

prt = 'の'
final_no, final_labels_no = copied_together(everything_no,all_labels_no,all_prt,prt)

#prt = 'Null'
#final_null, final_labels_NULL = copied_together(everything_NULL,all_labels_NULL,all_prt,prt)

#prt = 'Other'
#final_other, final_labels_other = copied_together(everything_other,all_labels_other,all_prt,prt)


In [24]:
prt='で'#change 'everything_*' and final_* below
part = 'de' #used for file name 
part_cap = 'De' #used for file name


train_set = everything_de+ final_mo[prt]+final_to[prt]+final_o[prt]+final_ga[prt]+final_ni[prt]+final_no[prt]+final_wa[prt]
print(len(everything_de))
print(len(train_set))

train_labels = all_labels_de+final_labels_mo[prt]+final_labels_to[prt]+final_labels_o[prt]+final_labels_ga[prt]+final_labels_ni[prt]+final_labels_no[prt]+final_labels_wa[prt]

new_train_labels = []
for item in train_labels:
    new_train_labels.append(str(item))
train_labels=new_train_labels

687
2641


# Development set

In [25]:
##dev

path = '/home/mhiraga/CorpusData_New/Corpus_Data/experiment/dev_and_test/*'
dev_everything, dev_all_labels = put_together_1(path,prt)    
print(len(dev_everything), len(dev_all_labels))


new_dev_all_labels = []
for item in dev_all_labels:
    new_dev_all_labels.append(str(item))
    
#print(new_dev_all_labels[:10])
dev_all_labels=new_dev_all_labels



173 173


# Check Class Distributions

In [26]:
import collections

le = preprocessing.LabelEncoder()
le.fit(train_labels)
#print(list(le.classes_))
counter=collections.Counter(train_labels)
allcount = 0

newcount={k: v for k, v in sorted(counter.items(), key=lambda item: item[1], reverse=True)}
for label,count in newcount.items():
    print(label[1:-1], '&', count, '&', "%0.2f" %(count*100/len(train_labels)), '\\\\')
    allcount += count
print('total: & ', allcount, '\\\\')

'Loc', 'COR' & 374 & 14.16 \\
'Time', 'INC' & 297 & 11.25 \\
'Toward', 'INC' & 294 & 11.13 \\
'MakeAdv', 'INC' & 253 & 9.58 \\
'StillLoc', 'INC' & 245 & 9.28 \\
'NiChange', 'INC' & 244 & 9.24 \\
'NiID', 'INC' & 189 & 7.16 \\
'IO', 'INC' & 144 & 5.45 \\
'Instr', 'COR' & 71 & 2.69 \\
'Agent', 'INC' & 68 & 2.57 \\
'Unknown', 'NULL', 'INC' & 63 & 2.39 \\
'Cause', 'COR' & 45 & 1.70 \\
'Duration-Completion', 'COR' & 35 & 1.33 \\
'Unknown', 'INC' & 35 & 1.33 \\
'Surface', 'INC' & 33 & 1.25 \\
'NiStinulus', 'INC' & 32 & 1.21 \\
'Unknown', 'Other', 'INC' & 29 & 1.10 \\
'NiReciprocal', 'INC' & 28 & 1.06 \\
'ModeOfAgent', 'COR' & 27 & 1.02 \\
'NiFor', 'INC' & 20 & 0.76 \\
'Unknown', 'COR' & 19 & 0.72 \\
'Appear/Manner', 'INC' & 19 & 0.72 \\
'Lang', 'COR' & 15 & 0.57 \\
'Unknown', 'De', 'INC' & 9 & 0.34 \\
'TimeExpression', 'INC' & 7 & 0.27 \\
'NiAbout', 'INC' & 6 & 0.23 \\
'COR' & 5 & 0.19 \\
'DO', 'INC' & 3 & 0.11 \\
'Material', 'COR' & 3 & 0.11 \\
'Attrib', 'INC' & 3 & 0.11 \\
'OtherP', 'INC' &

# Run classification

In [29]:
""" #used for model selection
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
"""
####################
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import precision_recall_fscore_support
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE# , BorderlineSMOTE, SVMSMOTE, SMOTENC
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.base import BaseSampler
from sklearn import metrics
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
pd.set_option('display.max_columns', None)

le = preprocessing.LabelEncoder()

def countLabel(list1):
    set1 = set(list1)

def fscoreToDF(fscore):
    result = []
    for x in fscore:
        result.append(x)
    fscoredf = pd.DataFrame(result, columns=reslist,index=['P','R','F','Total'])
    #fscoredf.index=['label','P','R','F','Total']
    #fscoredf.index.name = 'label'
    return fscoredf
    
def labelCompare(list1, list2):
    correctdict = {}
    incorrectdict = {}
    for count in range(0,len(list1)):
        if list1[count] == list2[count]:
            if list1[count] not in correctdict:
                correctdict[list1[count]] = 1
            else:
                correctdict[list1[count]] = correctdict[list1[count]]+1
        else:
            if list1[count] not in incorrectdict:
                incorrectdict[list1[count]]=1
            else:
                incorrectdict[list1[count]] = incorrectdict[list1[count]] +1
    #print('Right')
    outputlist = []
    for key, value in correctdict.items():
        output = {}
        if key in incorrectdict:
            inco_count = incorrectdict[key]
        else:
            inco_count = 0
        total = value + inco_count
        acc = value/total
        #output['label'] = key
        output['No.Right'] = value
        output['No.Wrong'] = inco_count
        output['Total'] = total
        output['Accuracy'] = acc
        output['label'] = str(key)
        outputlist.append(output)
        #print(key, ',', value, ',', inco_count, ',', total, ',', acc)
            
   
    for key, value in incorrectdict.items(): 
        output={}
        #print(key, ',', value)
        if key not in correctdict:
            co_count = 0
            total = value + co_count
            acc = co_count/total
            #output['label'] = key
            output['No.Right'] = co_count
            output['No.Wrong'] = value
            output['Total'] = total
            output['Accuracy'] = acc
            output['label'] = str(key)
            outputlist.append(output)
        else:
            co_count = correctdict[key]
            
    res = pd.DataFrame(outputlist)
    res = res[['label','Accuracy','Total','No.Right','No.Wrong']]
    
    return res

def labelEncode(final_labels):      
    le = preprocessing.LabelEncoder()

    final_labels_encoded = []
    for item in final_labels:
        final_labels_encoded.append(str(item))
    le.fit(final_labels_encoded)   
    return final_labels_encoded, le.classes_

    
def print_features(v):
    print("Number of Features: {} \\\\".format(len(v.get_feature_names())))
    features = v.get_feature_names()
    feature_set = set()
    for i in range(0,len(features)):
        m = features[i].find("=")
        feature_set.add(features[i][0:m])
    
    print(feature_set)

def print_prf(dev_array,predicted):
    scores = precision_recall_fscore_support(dev_array,predicted, average='weighted')
    scores = [n for n in scores]
    for i in range(0,len(scores)-1):
        if i == len(scores)-2:
            print(scores[i]*100, '\\\\')
        else:
            print(scores[i]*100, '&', end=' ')

def print_fscore(fscore_average): #tuple
    fscore_list = list(fscore_average)
    print("Total Weighted & ")
    for i in range(len(fscore_list)-1):
        new_score = (fscore_list[i])*100
        print("%0.1f" % new_score, end=" &")
    print("")    


In [32]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
v = DictVectorizer()


X_train = v.fit_transform(train_set)

print("Number of Features: {} \\\\".format(len(v.get_feature_names())))
print_features(v)


X_dev = v.transform(dev_everything)
print('\n')

clf = LogisticRegression(multi_class='ovr',max_iter=2000)
print('X_train shape: {} \\\\'.format(X_train.shape))


#y_train = MultiLabelBinarizer().fit_transform(final_labels_ni)
y_train = labelEncode(train_labels)[0]
clf.fit(X_train,y_train)
predicted = clf.predict(X_dev)



dev_array = np.asarray(dev_all_labels)


y_dev = labelEncode(dev_all_labels)[0]
print('Logistic Regression \\\\')
print('X_dev shape: {} \\\\'.format(X_dev.shape))
print('Accuracy score: {} \\\\'.format(clf.score(X_dev,y_dev)))


res = labelCompare(y_dev, predicted)

res.to_csv(path_or_buf="/home/mhiraga/CorpusData_New/Corpus_Data/multiclass_results/Ni_AllResults_latex/Generated_LogReg_ni",sep='&')



reslist = list(labelEncode(dev_all_labels)[1])

print('number of labels: {} \\\\'.format(len(reslist)))
print('number of each class: {} \\\\'.format(Counter(reslist)))
print(reslist)
#F1 score
fscore = precision_recall_fscore_support(dev_array,predicted,labels=reslist)
print(fscore)
#F0.5 score
fscore_point5 = precision_recall_fscore_support(dev_array,predicted,labels=reslist,beta=0.5)

# put them in df
fscoredf = fscoreToDF(fscore)
fscoredf = fscoredf.round(3).transpose()
fscoredf.columns = ['P','R','F1','Total']
fscoredf= fscoredf.rename_axis('label', axis=1)
fscoredf[['P','R','F1']] = fscoredf[['P','R','F1']].multiply(100)
fscoredf[['Total']] = fscoredf[['Total']].astype(int)
fscoredf = fscoredf.sort_values(by='Total',ascending=False)

print(fscoredf.to_latex())
fscore_average_ovs = precision_recall_fscore_support(dev_array,predicted,average='weighted')
print_fscore(fscore_average_ovs)

print('##########################')
print('random oversampling')
print('##########################')

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X_train, train_labels)
print(X_res.shape, "\\\\")
print('Resampled dataset shape %s \\\\' % Counter(y_res))

pipe = make_pipeline(RandomOverSampler(random_state=42), LogisticRegression(multi_class='ovr',max_iter=2000))
pipe.fit(X_train, train_labels)
predicted_1 = pipe.predict(X_dev)


dev_array = np.asarray(dev_all_labels)
#print(dev_array)
print('LR: RandomOverSampling \\\\')
res_ov = labelCompare(dev_array, predicted_1)
print('Accuracy score: {} \\\\'.format(pipe.score(X_dev,dev_array)))
print('number of labels: {} \\\\'.format(len(reslist)))

fscore_ov = precision_recall_fscore_support(dev_array,predicted_1,labels=reslist)
fscoredf_ov = fscoreToDF(fscore_ov)
fscoredf_ov = fscoredf_ov.round(3).transpose()
fscoredf_ov.columns = ['P','R','F1','Total']
fscoredf_ov[['P','R','F1']] = fscoredf_ov[['P','R','F1']]*100
fscoredf_ov= fscoredf_ov.rename_axis('label', axis=1)
fscoredf[['Total']] = fscoredf[['Total']].astype(int)
fscoredf = fscoredf.sort_values(by='Total',ascending=False)

print("\n\n")
print("########################Oversampling Table#####################")
print(fscoredf_ov.to_latex())
fscore_average_ovs = precision_recall_fscore_support(dev_array,predicted_1,average='weighted')
print_fscore(fscore_average_ovs)

print('#####################\n\n')
#fit(X_train, all_labels)





###################




Number of Features: 8231 \\
Number of Features: 8231 \\
{'word-2', 'Verbs_0', 'word+2', 'Verbs_4', 'OtherPrt_0', 'Verbs_7', 'lemma+1', 'OtherPrt_3', 'POS+2', 'POS-3', 'dep_head', 'Verbs_1', 'word-1', 'Verbs_2', 'Verbs_10', 'word-3', 'Verbs_6', 'POS+3', 'Verbs_9', 'POS-2', 'word+1', 'Verbs_3', 'Verbs_5', 'OtherPrt_2', 'POS-1', 'POS+1', 'Verbs_8', 'word+3', 'OtherPrt_1', 'lemma-1', 'target'}


X_train shape: (2641, 8231) \\
Logistic Regression \\
X_dev shape: (173, 8231) \\
Accuracy score: 0.653179190751445 \\
number of labels: 19 \\
number of each class: Counter({"['Attrib', 'INC']": 1, "['Cause', 'COR']": 1, "['DO', 'INC']": 1, "['Duration-Completion', 'COR']": 1, "['Even', 'INC']": 1, "['Instr', 'COR']": 1, "['Lang', 'COR']": 1, "['Loc', 'COR']": 1, "['MakeAdv', 'INC']": 1, "['Material', 'COR']": 1, "['ModeOfAgent', 'COR']": 1, "['OtherP', 'INC']": 1, "['StillLoc', 'COR']": 1, "['StillLoc', 'INC']": 1, "['Surface', 'INC']": 1, "['Time', 'INC']": 1, "['Toward', 'INC']": 1, "['Unknown',

/home/mhiraga/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mhiraga/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mhiraga/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LR: RandomOverSampling \\
Accuracy score: 0.6763005780346821 \\
number of labels: 19 \\



########################Oversampling Table#####################
\begin{tabular}{lrrrr}
\toprule
label &      P &     R &    F1 &  Total \\
\midrule
['Attrib', 'INC']              &    0.0 &   0.0 &   0.0 &    1.0 \\
['Cause', 'COR']               &  100.0 &  33.3 &  50.0 &   15.0 \\
['DO', 'INC']                  &    0.0 &   0.0 &   0.0 &    2.0 \\
['Duration-Completion', 'COR'] &   62.5 &  62.5 &  62.5 &    8.0 \\
['Even', 'INC']                &    0.0 &   0.0 &   0.0 &    1.0 \\
['Instr', 'COR']               &   33.3 &  44.4 &  38.1 &    9.0 \\
['Lang', 'COR']                &    0.0 &   0.0 &   0.0 &    1.0 \\
['Loc', 'COR']                 &   69.6 &  96.9 &  81.0 &   97.0 \\
['MakeAdv', 'INC']             &    0.0 &   0.0 &   0.0 &    1.0 \\
['Material', 'COR']            &    0.0 &   0.0 &   0.0 &    1.0 \\
['ModeOfAgent', 'COR']         &   87.5 &  70.0 &  77.8 &   10.0 \\
['OtherP', 'I

/home/mhiraga/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mhiraga/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mhiraga/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
###########################
### merge two dfs f1
###########################


fscoredf = fscoredf.drop(['Total'],axis=1)

frame = [fscoredf, fscoredf_ov]

result_ov = pd.concat(frame,axis=1)

result_ov['Total'] = result_ov['Total'].astype(int)

result_ov = result_ov.sort_values(by='Total',ascending=False)

print('##############FINAL TABLE################################')
print(result_ov)
print('\n\n Latex table')
print(result_ov.to_latex())
print('#####################\n\n')

##############FINAL TABLE################################
label                               P     R    F1      P     R    F1  Total
['Loc', 'COR']                   64.6  97.9  77.9   69.6  96.9  81.0     97
['Cause', 'COR']                100.0  20.0  33.3  100.0  33.3  50.0     15
['StillLoc', 'INC']             100.0  16.7  28.6  100.0  16.7  28.6     12
['ModeOfAgent', 'COR']           85.7  60.0  70.6   87.5  70.0  77.8     10
['Instr', 'COR']                 37.5  33.3  35.3   33.3  44.4  38.1      9
['Duration-Completion', 'COR']   66.7  50.0  57.1   62.5  62.5  62.5      8
['Unknown', 'COR']                0.0   0.0   0.0    0.0   0.0   0.0      8
['DO', 'INC']                     0.0   0.0   0.0    0.0   0.0   0.0      2
['Time', 'INC']                   0.0   0.0   0.0    0.0   0.0   0.0      2
['StillLoc', 'COR']               0.0   0.0   0.0    0.0   0.0   0.0      1
['Toward', 'INC']                 0.0   0.0   0.0    0.0   0.0   0.0      1
['Surface', 'INC']            

In [38]:

print("\n\n######################\n")
print("Synthetic Data w/o over sampling vs w/ oversampling")
#result_both.index = result_both.index.map(lambda x: str(x))
result_ov.index = result_ov.index.map(lambda x: str(x))


print(result_ov.to_latex())
output_path = '/home/mhiraga/CorpusData_New/Corpus_Data/multiclass_results/{}_AllResults_latex/RandomOverSampling_{}'.format(part_cap, part)
result_ov.to_csv(path_or_buf= output_path,sep='&')




######################

Synthetic Data w/o over sampling vs w/ oversampling
\begin{tabular}{lrrrrrrr}
\toprule
label &      P &     R &    F1 &      P &     R &    F1 &  Total \\
\midrule
['Loc', 'COR']                 &   64.6 &  97.9 &  77.9 &   69.6 &  96.9 &  81.0 &     97 \\
['Cause', 'COR']               &  100.0 &  20.0 &  33.3 &  100.0 &  33.3 &  50.0 &     15 \\
['StillLoc', 'INC']            &  100.0 &  16.7 &  28.6 &  100.0 &  16.7 &  28.6 &     12 \\
['ModeOfAgent', 'COR']         &   85.7 &  60.0 &  70.6 &   87.5 &  70.0 &  77.8 &     10 \\
['Instr', 'COR']               &   37.5 &  33.3 &  35.3 &   33.3 &  44.4 &  38.1 &      9 \\
['Duration-Completion', 'COR'] &   66.7 &  50.0 &  57.1 &   62.5 &  62.5 &  62.5 &      8 \\
['Unknown', 'COR']             &    0.0 &   0.0 &   0.0 &    0.0 &   0.0 &   0.0 &      8 \\
['DO', 'INC']                  &    0.0 &   0.0 &   0.0 &    0.0 &   0.0 &   0.0 &      2 \\
['Time', 'INC']                &    0.0 &   0.0 &   0.0 &    0.0 &

In [39]:
## load file from Classification experiment and merge
baseline_path = '/home/mhiraga/CorpusData_New/Corpus_Data/multiclass_results/{}_AllResults_latex/BaselineOversampling_{}'.format(part_cap, part)
baseline = pd.read_csv(baseline_path ,sep='&')
print(baseline)

generated_path = '/home/mhiraga/CorpusData_New/Corpus_Data/multiclass_results/{}_AllResults_latex/RandomOverSampling_{}'.format(part_cap, part)
generated_oversample = pd.read_csv(generated_path,sep='&')
print(generated_oversample)

all_together = baseline.merge(generated_oversample, how = 'inner', on = 'Unnamed: 0')
all_together['Total'] = all_together['Total'].fillna(0.0).astype(int)
all_together = all_together.set_index('Unnamed: 0')
print(all_together.to_latex())

all_together_output = '/home/mhiraga/CorpusData_New/Corpus_Data/multiclass_results/{}_AllResults_latex/AllTogetherLatex_{}'.format(part_cap, part)
all_together.to_csv(path_or_buf=all_together_output,sep='&')


                       Unnamed: 0      P     R    F1    P.1   R.1  F1.1
0                  ['Loc', 'COR']   68.3  97.9  80.5   72.8  93.8  82.0
1          ['N', '/', 'A', 'INC']   62.5  21.7  32.3   50.0  30.4  37.8
2                ['Cause', 'COR']  100.0  26.7  42.1   80.0  26.7  40.0
3          ['ModeOfAgent', 'COR']   87.5  70.0  77.8   87.5  70.0  77.8
4                ['Instr', 'COR']   37.5  33.3  35.3   27.3  33.3  30.0
5  ['Duration-Completion', 'COR']   66.7  50.0  57.1   66.7  75.0  70.6
6              ['Unknown', 'COR']    0.0   0.0   0.0  100.0  12.5  22.2
7             ['StillLoc', 'COR']    0.0   0.0   0.0    0.0   0.0   0.0
8             ['Material', 'COR']    0.0   0.0   0.0    0.0   0.0   0.0
9                 ['Lang', 'COR']    0.0   0.0   0.0    0.0   0.0   0.0
                        Unnamed: 0      P     R    F1    P.1   R.1  F1.1  \
0                   ['Loc', 'COR']   64.6  97.9  77.9   69.6  96.9  81.0   
1                 ['Cause', 'COR']  100.0  20.0  33.3  1